In [9]:
import os
import cv2
import numpy as np

# === Paths ===
canvas_path = "input_flag.png"
apple_dir = "apple_images"

# === Step 1. Load canvas ===
canvas = cv2.imread(canvas_path)
canvas_gray = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
canvas_h, canvas_w = canvas_gray.shape

# === Step 2. Create binary masks for all apples ===
mask_list = []
for file in os.listdir(apple_dir):
    if file.lower().endswith((".png", ".jpg", ".jpeg")):
        img_path = os.path.join(apple_dir, file)
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Apples are white, background is black
        _, mask = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY)
        mask_list.append(mask)

# === Step 3. Combine all masks ===
if mask_list:
    max_height = max(m.shape[0] for m in mask_list)
    max_width  = max(m.shape[1] for m in mask_list)
    combined_mask = np.zeros((max_height, max_width), dtype=np.uint8)
    for m in mask_list:
        if m.shape != combined_mask.shape:
            m_resized = cv2.resize(m, (max_width, max_height))
        else:
            m_resized = m
        combined_mask = cv2.bitwise_or(combined_mask, m_resized)
else:
    raise ValueError("No apple masks found!")

# === Step 4. Resize mask to canvas size ===
mask_resized = cv2.resize(combined_mask, (canvas_w, canvas_h))
mask_bin = (mask_resized > 128).astype(np.uint8)  # apples=1, background=0

# === Step 5. Apply mask to canvas ===
# We want to keep letters visible where mask=0
result = canvas_gray.copy()
result[mask_bin == 1] = 255  # set apple regions to white (letters hidden)

# Save for inspection
cv2.imwrite("flag_visible_correct.png", result)

print("✅ Result saved as 'flag_visible_correct.png'.")
print("💡 Open the image — only letters not covered by apples remain. Those letters form the flag.")


✅ Result saved as 'flag_visible_correct.png'.
💡 Open the image — only letters not covered by apples remain. Those letters form the flag.


In [13]:
import cv2
import numpy as np

# Load masked grid
img = cv2.imread("flag_visible_grid.png", cv2.IMREAD_GRAYSCALE)
h, w = img.shape

# Letters grid (rows must match the canvas)
letters = [
    list("IX_BKFCKAUPRAIOTI"),
    list("EJDUCBTMWYVGOSBXC"),
    list("YQPAIZYZTMBRDEZAYR"),
    list("SBDNLHEMISPHEREXUM"),
    list("AHRSZORBXEJIVUGKTR"),
    list("MKEQKD_SMFZPJEVNBK"),
    list("XZVOPJXWLBIDOHMVRS"),
    list("MXTRNFIJBCGRWPIXFY"),
    list("HNXRV_DLKZGTIAGVTH"),
    list("XRPJYDTIUOWBCZLKIE"),
    list("OZVEBIWJCDSHXMNUR"),
    list("LAEWDSLNVZI_BUFJSK"),
    list("UFCO_KHXFGRDNBYVEO"),
    list("LXZJWARNQIOPNSELYB"),
    list("_HKBAYERILSJNAUTVR"),
    list("IRSLWGOKTFPNPPMIOK"),
    list("WXHTJKLELUAUBZSCRD"),
    list("GVYEZFMSJKTBHRVPMG")
]

rows = len(letters)
cell_h = h // rows

flag = ""

for i, row in enumerate(letters):
    cols = len(row)
    cell_w = w // cols
    for j, ch in enumerate(row):
        y0, y1 = i*cell_h, (i+1)*cell_h
        x0, x1 = j*cell_w, (j+1)*cell_w

        cell = img[y0:y1, x0:x1]

        if np.mean(cell) < 200:  # dark = visible letter
            flag += ch

print("🚩 FLAG:", flag)

🚩 FLAG: IX_BKFCKAUPRAIOTIEJDUCBTMWYVGOSBXCYQPAIZYZTMBRDEZAYRSBDNLHEMISPHEREXUMAHRSZORBXEJIVUGKTRMKEQKD_SMFZPJEVNBKXZVOPJXWLBIDOHMVRSMXTRNFIJBCGRWPIXFYHNXRV_DLKZGTIAGVTHXRPJYDTIUOWBCZLKIEOZVEBIWJCDSHXMNURLAEWDSLNVZI_BUFJSKUFCO_KHXFGRDNBYVEOLXZJWARNQIOPNSELYB_HKBAYERILSJNAUTVRIRSLWGOKTFPNPPMIOKWXHTJKLELUAUBZSCRDGVYEZFMSJKTBHRVPMG


In [14]:
import os
import cv2
import numpy as np

# === Paths ===
canvas_path = "input_flag.png"
apple_dir = "apple_images"

# === Step 1. Load canvas ===
canvas = cv2.imread(canvas_path)
canvas_gray = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
canvas_h, canvas_w = canvas_gray.shape

# === Step 2. Create binary masks for all apples ===
mask_list = []
for file in os.listdir(apple_dir):
    if file.lower().endswith((".png", ".jpg", ".jpeg")):
        img_path = os.path.join(apple_dir, file)
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Apples are white, background is black
        _, mask = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY)
        mask_list.append(mask)

# === Step 3. Combine all masks ===
if mask_list:
    max_height = max(m.shape[0] for m in mask_list)
    max_width  = max(m.shape[1] for m in mask_list)
    combined_mask = np.zeros((max_height, max_width), dtype=np.uint8)
    for m in mask_list:
        if m.shape != combined_mask.shape:
            m_resized = cv2.resize(m, (max_width, max_height))
        else:
            m_resized = m
        combined_mask = cv2.bitwise_or(combined_mask, m_resized)
else:
    raise ValueError("No apple masks found!")

# === Step 4. Resize mask to canvas size ===
mask_resized = cv2.resize(combined_mask, (canvas_w, canvas_h))
mask_bin = (mask_resized > 128).astype(np.uint8)  # apples=1, background=0

# === Step 5. Apply mask to canvas ===
# Letters under apples become white (hidden), others remain visible
masked_canvas = canvas_gray.copy()
masked_canvas[mask_bin == 1] = 255
cv2.imwrite("flag_visible_correct.png", masked_canvas)

# === Step 6. Define the letter grid (from original input_flag.png) ===
letters = [
    list("IX_BKFCKAUPRAIOTI"),
    list("EJDUCBTMWYVGOSBXC"),
    list("YQPAIZYZTMBRDEZAYR"),
    list("SBDNLHEMISPHEREXUM"),
    list("AHRSZORBXEJIVUGKTR"),
    list("MKEQKD_SMFZPJEVNBK"),
    list("XZVOPJXWLBIDOHMVRS"),
    list("MXTRNFIJBCGRWPIXFY"),
    list("HNXRV_DLKZGTIAGVTH"),
    list("XRPJYDTIUOWBCZLKIE"),
    list("OZVEBIWJCDSHXMNUR"),
    list("LAEWDSLNVZI_BUFJSK"),
    list("UFCO_KHXFGRDNBYVEO"),
    list("LXZJWARNQIOPNSELYB"),
    list("_HKBAYERILSJNAUTVR"),
    list("IRSLWGOKTFPNPPMIOK"),
    list("WXHTJKLELUAUBZSCRD"),
    list("GVYEZFMSJKTBHRVPMG")
]

# === Step 7. Extract visible letters automatically ===
flag = ""
rows = len(letters)
cell_h = canvas_h // rows

for i, row in enumerate(letters):
    cols = len(row)
    cell_w = canvas_w // cols
    for j, ch in enumerate(row):
        y0, y1 = i*cell_h, (i+1)*cell_h
        x0, x1 = j*cell_w, (j+1)*cell_w

        cell = masked_canvas[y0:y1, x0:x1]

        # If the letter is visible (dark enough)
        if np.mean(cell) < 200:
            flag += ch

print("🚩 FLAG:", flag)
print("✅ Image saved as 'flag_visible_correct.png'.")

🚩 FLAG: 
✅ Image saved as 'flag_visible_correct.png'.


In [15]:
import os
import cv2
import numpy as np

# === Paths ===
canvas_path = "input_flag.png"
apple_dir = "apple_images"

# === Step 1. Load canvas ===
canvas = cv2.imread(canvas_path)
canvas_gray = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
canvas_h, canvas_w = canvas_gray.shape

# === Step 2. Create binary masks for all apples ===
mask_list = []
for file in os.listdir(apple_dir):
    if file.lower().endswith((".png", ".jpg", ".jpeg")):
        img_path = os.path.join(apple_dir, file)
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, mask = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY)
        mask_list.append(mask)

# === Step 3. Combine all masks ===
if mask_list:
    max_height = max(m.shape[0] for m in mask_list)
    max_width  = max(m.shape[1] for m in mask_list)
    combined_mask = np.zeros((max_height, max_width), dtype=np.uint8)
    for m in mask_list:
        if m.shape != combined_mask.shape:
            m_resized = cv2.resize(m, (max_width, max_height))
        else:
            m_resized = m
        combined_mask = cv2.bitwise_or(combined_mask, m_resized)
else:
    raise ValueError("No apple masks found!")

# === Step 4. Resize mask to canvas size ===
mask_resized = cv2.resize(combined_mask, (canvas_w, canvas_h))
mask_bin = (mask_resized > 128).astype(np.uint8)  # apples=1, background=0

# === Step 5. Apply mask to canvas ===
masked_canvas = canvas_gray.copy()
masked_canvas[mask_bin == 1] = 255  # hide apple areas
cv2.imwrite("flag_visible_correct.png", masked_canvas)

# === Step 6. Define the letter grid ===
letters = [
    list("IX_BKFCKAUPRAIOTI"),
    list("EJDUCBTMWYVGOSBXC"),
    list("YQPAIZYZTMBRDEZAYR"),
    list("SBDNLHEMISPHEREXUM"),
    list("AHRSZORBXEJIVUGKTR"),
    list("MKEQKD_SMFZPJEVNBK"),
    list("XZVOPJXWLBIDOHMVRS"),
    list("MXTRNFIJBCGRWPIXFY"),
    list("HNXRV_DLKZGTIAGVTH"),
    list("XRPJYDTIUOWBCZLKIE"),
    list("OZVEBIWJCDSHXMNUR"),
    list("LAEWDSLNVZI_BUFJSK"),
    list("UFCO_KHXFGRDNBYVEO"),
    list("LXZJWARNQIOPNSELYB"),
    list("_HKBAYERILSJNAUTVR"),
    list("IRSLWGOKTFPNPPMIOK"),
    list("WXHTJKLELUAUBZSCRD"),
    list("GVYEZFMSJKTBHRVPMG")
]

# === Step 7. Extract visible letters and highlight ===
rows = len(letters)
cell_h = canvas_h // rows

# Create color version for visualization
canvas_color = cv2.cvtColor(canvas_gray, cv2.COLOR_GRAY2BGR)
flag = ""

for i, row in enumerate(letters):
    cols = len(row)
    cell_w = canvas_w // cols
    for j, ch in enumerate(row):
        y0, y1 = i*cell_h, (i+1)*cell_h
        x0, x1 = j*cell_w, (j+1)*cell_w
        cell = masked_canvas[y0:y1, x0:x1]

        # Dynamic threshold
        threshold = 0.9 * np.max(cell)
        if np.min(cell) < threshold:
            flag += ch
            # Highlight the letter in red
            cv2.rectangle(canvas_color, (x0, y0), (x1, y1), (0, 0, 255), 2)

# === Step 8. Save the highlighted visualization ===
cv2.imwrite("flag_highlighted.png", canvas_color)

print("🚩 FLAG:", flag)
print("✅ Highlighted image saved as 'flag_highlighted.png'.")
print("💡 Red boxes show exactly which letters form the flag.")

🚩 FLAG: OTIXCYQAYBERUMHRSVUGKTRQKVNBKXZOPJXMVRSMXFYVTHIESHRBUO_KHXGRDNLXZJWARNIOPL_KBAYRILJTVITFPNPPIOKEUBZSCRJHPMG
✅ Highlighted image saved as 'flag_highlighted.png'.
💡 Red boxes show exactly which letters form the flag.


In [22]:
import os
import cv2
import numpy as np

# === Paths ===
canvas_path = "input_flag.png"
apple_dir = "apple_images"

# === Step 1. Load canvas ===
canvas = cv2.imread(canvas_path)
canvas_gray = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
canvas_h, canvas_w = canvas_gray.shape

# === Step 2. Create binary masks for all apples ===
mask_list = []
for file in os.listdir(apple_dir):
    if file.lower().endswith((".png", ".jpg", ".jpeg")):
        img_path = os.path.join(apple_dir, file)
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, mask = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY)
        mask_list.append(mask)

# === Step 3. Combine all masks ===
if mask_list:
    max_height = max(m.shape[0] for m in mask_list)
    max_width  = max(m.shape[1] for m in mask_list)
    combined_mask = np.zeros((max_height, max_width), dtype=np.uint8)
    for m in mask_list:
        if m.shape != combined_mask.shape:
            m_resized = cv2.resize(m, (max_width, max_height))
        else:
            m_resized = m
        combined_mask = cv2.bitwise_or(combined_mask, m_resized)
else:
    raise ValueError("No apple masks found!")

# === Step 4. Resize mask to canvas size ===
mask_resized = cv2.resize(combined_mask, (canvas_w, canvas_h))
mask_bin = (mask_resized > 128).astype(np.uint8)  # apples=1

# === Step 5. Define the letter grid with manually replaced letters ===
letters = [
    list("IXBBKFCKAUPRAIOTI"),  # row 0: changed _ → B at index 2
    list("EJDUCBTMWYVGOSBXC"),
    list("YQPAIZYZTMBRDEZAYR"),
    list("SBDNLHEMISPHEREXUM"),
    list("AHRSZORBXEJIVUGKTR"),
    list("MKEQKDSSMFZPJEVNBK"),  # row 5: changed _ → S at index 6
    list("XZVOPJXWLBIDOHMVRS"),
    list("MXTRNFIJBCGRWPIXFY"),
    list("HNXRVDDLKZGTIAGVTH"),  # row 8: changed _ → D at index 5
    list("XRPJYDTIUOWBCZLKIE"),
    list("OZVEBIWJCDSHXMNUR"),
    list("LAEWDSLNVZIUBUFJSK"),  # row 11: changed _ → U at index 11
    list("UFCOUKHXFGRDNBYVEO"),  # row 12: changed _ → K at index 4
    list("LXZJWARNQIOPNSELYB"),
    list("HHKBAYERILSJNAUTVR"),  # row 14: changed _ → H at index 0
    list("IRSLWGOKTFPNPPMIOK"),
    list("WXHTJKLELUAUBZSCRD"),
    list("GVYEZFMSJKTBHRVPMG")
]

# === Step 6. Extract letters at known positions under apples ===
rows = len(letters)
cell_h = canvas_h // rows
flag = ""

# These are the known positions of the original underscores
target_coords = [(0, 2), (5, 6), (8, 5), (11, 11), (12, 4), (14, 0)]

for i, j in target_coords:
    row = letters[i]
    cols = len(row)
    cell_w = canvas_w // cols
    y0, y1 = i * cell_h, (i + 1) * cell_h
    x0, x1 = j * cell_w, (j + 1) * cell_w
    cell_mask = mask_bin[y0:y1, x0:x1]
    if np.any(cell_mask == 1):
        cell_img = canvas_gray[y0:y1, x0:x1]
        letter_pixel = np.min(cell_img)
        if letter_pixel < 230:
            flag += row[j]
            print(f"Found letter at ({i},{j}): {row[j]}")

print("🚩 FLAG:", flag)

Found letter at (0,2): B
Found letter at (5,6): S
Found letter at (8,5): D
Found letter at (11,11): U
Found letter at (12,4): U
Found letter at (14,0): H
🚩 FLAG: BSDUUH


In [24]:
import os
import cv2
import numpy as np
from collections import defaultdict

# === Paths ===
canvas_path = "input_flag.png"
apple_dir = "apple_images"

# === Step 1. Load canvas ===
canvas = cv2.imread(canvas_path)
if canvas is None:
    raise FileNotFoundError(f"Cannot find '{canvas_path}'")
gray = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
H, W = gray.shape

# === Step 2. Define the letter grid ===
letters = [
    list("IX_BKFCKAUPRAIOTI"),
    list("EJDUCBTMWYVGOSBXC"),
    list("YQPAIZYZTMBRDEZAYR"),
    list("SBDNLHEMISPHEREXUM"),
    list("AHRSZORBXEJIVUGKTR"),
    list("MKEQKD_SMFZPJEVNBK"),
    list("XZVOPJXWLBIDOHMVRS"),
    list("MXTRNFIJBCGRWPIXFY"),
    list("HNXRV_DLKZGTIAGVTH"),
    list("XRPJYDTIUOWBCZLKIE"),
    list("OZVEBIWJCDSHXMNUR"),
    list("LAEWDSLNVZI_BUFJSK"),
    list("UFCO_KHXFGRDNBYVEO"),
    list("LXZJWARNQIOPNSELYB"),
    list("_HKBAYERILSJNAUTVR"),
    list("IRSLWGOKTFPNPPMIOK"),
    list("WXHTJKLELUAUBZSCRD"),
    list("GVYEZFMSJKTBHRVPMG")
]

rows = len(letters)
row_cols = [len(r) for r in letters]
cell_h = H // rows

# === Step 3. Crop helper ===
def crop_cell(img, i, j):
    cols = row_cols[i]
    cw = W // cols
    y0, y1 = i*cell_h, (i+1)*cell_h
    x0, x1 = j*cw, (j+1)*cw
    return img[y0:y1, x0:x1]

# === Step 4. Preprocess cells for template matching ===
def preprocess_cell(cell):
    c = cv2.resize(cell, (64,64))
    th = cv2.adaptiveThreshold(c, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY_INV, 11, 2)
    return th

# === Step 5. Build templates from non-underscore letters ===
templates = defaultdict(list)  # letter -> list of images
for i,row in enumerate(letters):
    for j,ch in enumerate(row):
        if ch == "_":
            continue
        cell = crop_cell(gray, i, j)
        proc = preprocess_cell(cell)
        templates[ch].append(proc)

# === Step 6. Match underscore cells to templates ===
flag_letters = []

for i,row in enumerate(letters):
    cols = row_cols[i]
    for j,ch in enumerate(row):
        if ch != "_":
            continue
        cell = crop_cell(gray, i, j)
        proc_cell = preprocess_cell(cell)
        best_letter = None
        best_score = -1.0
        for letter, t_list in templates.items():
            for t in t_list:
                res = cv2.matchTemplate(proc_cell, t, cv2.TM_CCOEFF_NORMED)
                score = res.max()
                if score > best_score:
                    best_score = score
                    best_letter = letter
        flag_letters.append((i,j,best_letter,best_score))

# === Step 7. Sort letters by reading order and form flag ===
flag_letters_sorted = sorted(flag_letters, key=lambda x: (x[0], x[1]))
flag = "".join([f[2] for f in flag_letters_sorted])

print("🚩 FLAG:", flag)

# === Optional: Annotate canvas with detected flag letters ===
canvas_vis = canvas.copy()
font = cv2.FONT_HERSHEY_SIMPLEX
for i,j,letter,score in flag_letters_sorted:
    cols = row_cols[i]
    cw = W // cols
    x0, y0 = j*cw, i*cell_h
    x1, y1 = x0 + cw, y0 + cell_h
    cv2.rectangle(canvas_vis, (x0, y0), (x1, y1), (0,255,0), 2)
    cv2.putText(canvas_vis, letter, (x0+5, y0+int(cell_h*0.7)),
                font, 0.7, (0,255,0), 2, cv2.LINE_AA)

cv2.imwrite("flag_detected.png", canvas_vis)
print("✅ Annotated flag image saved as 'flag_detected.png'")

🚩 FLAG: DEDIDL
✅ Annotated flag image saved as 'flag_detected.png'
